## post processing pipeline to convert geojson files

In [1]:
import geopandas as gp
from geopandas.tools import sjoin
import numpy as np
import pandas as pd
from shapely.geometry import Point

OSError: could not find or load spatialindex_c-64.dll

In [44]:
%%time
def evaluate(rtype, area):
'''Find intersection between predicted roof typologies and OSM building boundaries. 

Keeps the interecting polygons and outputs OSM building polygons that contain solar/green roofs

'''
    city = 'Final_Prediction/01GeoJson/' + area + '.geojson'
    city = gp.GeoDataFrame.from_file(city)[['geometry']]  
    city['area'] = city['geometry'].to_crs({'init': 'epsg:3395'}).map(lambda p: p.area)
    
    intersections = 'Final_Prediction/07Prediction/' + rtype + '/' + area + '_' + rtype + ".geojson"
    intersections = gp.GeoDataFrame.from_file(intersections) 

    bldg_count = city['area'].count()
    intersection_count = intersections['area'].count()
    percentage_count = intersection_count/bldg_count*100
    percentage_area = intersections['area'].sum()/city['area'].sum()*100
    
    df = pd.DataFrame([[area, bldg_count, intersection_count, percentage_count, percentage_area]], columns=['City','Buildings', rtype +' Roofs','%Count','%Area'])

    return df

Wall time: 0 ns


In [45]:
greenlist = []
for i in os.listdir('Final_Prediction/07Prediction/Green'):
    name = i[:-14]     
    greenlist.append(name)
print(greenlist)

solarlist = []
for i in os.listdir('Final_Prediction/07Prediction/Solar'):
    name = i[:-14]     
    solarlist.append(name)
print(solarlist)

['Berlin', 'Boston', 'Copenhagen', 'Las Vegas', 'Los Angeles', 'Melbourne', 'New York', 'Paris', 'Phoenix', 'Portland', 'San Diego', 'SanJose', 'Seattle', 'SF', 'Vancouver', 'Zurich']
['Berkeley', 'Berlin', 'Boston', 'Copenhagen', 'Las Vegas', 'Los Angeles', 'Melbourne', 'New York', 'Paris', 'Phoenix', 'Portland', 'San Diego', 'SanJose', 'Seattle', 'SF', 'Vancouver', 'Zurich']


In [46]:
# creating empty df
df_solar = pd.DataFrame([], columns=['City','Buildings', 'Solar Roofs','%Count','%Area'])
df_green = pd.DataFrame([], columns=['City','Buildings', 'Green Roofs','%Count','%Area'])

In [47]:
for area in solarlist:
    df_solar = df_solar.append(evaluate('Solar', area))

In [48]:
from sklearn.preprocessing import MinMaxScaler

In [49]:
# find percentage of distribution
min_max_scaler = MinMaxScaler()
solar_c_scaled = (min_max_scaler.fit_transform(df_solar[['%Count']])*100).astype('int') 
solar_a_scaled = (min_max_scaler.fit_transform(df_solar[['%Area']])*100).astype('int') 

In [50]:
df_solar['Ranking by %Count'] = solar_c_scaled
df_solar['Ranking by %Area'] = solar_a_scaled
df_solar['Overall Ranking'] = (df_solar['Ranking by %Count'] + df_solar['Ranking by %Area'])/2

In [51]:
df_solar = df_solar.sort_values(by=['Overall Ranking'], ascending=False)

In [52]:
df_solar.to_csv('Solar Results.csv')

In [53]:
for area in greenlist:
    df_green = df_green.append(evaluate('Green', area))

In [54]:
green_c_scaled = (min_max_scaler.fit_transform(df_green[['%Count']])*100).astype('int') 
green_a_scaled = (min_max_scaler.fit_transform(df_green[['%Area']])*100).astype('int') 

In [56]:
df_green['Ranking by %Count'] = green_c_scaled
df_green['Ranking by %Area'] = green_a_scaled
df_green['Overall Ranking'] = (df_green['Ranking by %Count'] + df_green['Ranking by %Area'])/2

In [58]:
df_green = df_green.sort_values(by=['Overall Ranking'], ascending=False)
df_green.to_csv('Green Results.csv')